In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


## Import data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True,as_supervised=True)

## Data Preprocessing

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_validation_samples, tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffle_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE) 

validation_data = shuffle_train_and_validation_data.take(num_validation_samples)

train_data = shuffle_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])


### Choose the optimizer and the loss function

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [6]:
NUM_EPOCH = 5

model.fit(train_data,epochs=NUM_EPOCH,validation_data =(validation_inputs, validation_targets), verbose=2) 

Epoch 1/5
540/540 - 3s - loss: 0.3201 - accuracy: 0.9081 - val_loss: 0.1531 - val_accuracy: 0.9553 - 3s/epoch - 5ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1283 - accuracy: 0.9621 - val_loss: 0.1117 - val_accuracy: 0.9667 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0935 - accuracy: 0.9714 - val_loss: 0.0831 - val_accuracy: 0.9757 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0709 - accuracy: 0.9779 - val_loss: 0.0809 - val_accuracy: 0.9765 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0572 - accuracy: 0.9826 - val_loss: 0.0585 - val_accuracy: 0.9822 - 2s/epoch - 3ms/step


## Test the model

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

2/2 [==============================] - 0s 92ms/step - loss: 0.0782 - accuracy: 0.9748


In [8]:
print('Test los {0:.2f}. Test accuracy {1:.2f}%'.format(test_loss,test_accuracy*100))

Test los 0.08. Test accuracy 97.48%
